In [6]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from PIL import Image, ImageFile
import re
import nltk
import os
import pickle

AttributeError: module 'numpy.random' has no attribute 'default_rng'

In [5]:
#!pip install Pillow

In [33]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [5]:
#albums_path = '/Users/Nick/OneDrive/College/2018 Fall/Data Science Lab EE460J/Final Project/output'
#output_path = '/Users/Nick/OneDrive/College/2018 Fall/Data Science Lab EE460J/Final Project/embeddings'

albums_path = '/home/shezin/Desktop/Lyristic/output'
output_path = '/home/shezin/Desktop/Lyristic/emb'

In [7]:
def format_data(albums_path):
    all_lyrics = []
    
    for dirName, subdirList, fileList in os.walk(albums_path, topdown=False):
        for fname in fileList:
            
            if fname.endswith('.txt'):
                lyrics_file = open(dirName + '/' + fname, "r")
                lyrics = lyrics_file.read()
                lyrics_file.close()
                
                
                lyrics = lyrics.replace('\n', ' ')
                lyrics = lyrics.replace('\\', '')
                lyrics = lyrics.strip(' ')
                #lyrics = lyrics.replace(r"\[.*\]","")
                lyrics = re.sub(r'\[[^\]]*\]', '', lyrics)
                all_lyrics.append(lyrics.lower())
                
                
            if fname.endswith('.jpg'):
                #Hidden file issues in colab
                resized_img = Image.open(dirName + '/' + fname).resize((256, 256))
                resized_img.save(dirName + '/' + fname)
            
    return all_lyrics

In [8]:
lyrics = format_data(albums_path)

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(lyrics)]

In [9]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 256
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1,
                epochs=5)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("newd2v.model")
print("new Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [10]:
# from gensim.models.doc2vec import Doc2Vec

# model= Doc2Vec.load("newd2v.model")

# #to find the vector of a document which is not in training data
# test_data = word_tokenize("Smells like teen spirit".lower())
# v1 = model.infer_vector(test_data)
# print("V1_infer", v1)

# # to find most similar doc using tags
# similar_doc = model.docvecs.most_similar('1')
# print(similar_doc)


# # to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
# print(model.docvecs['1'])

V1_infer [-1.92375656e-03 -2.54958346e-02 -1.96488239e-02  1.15390597e-02
  2.83209905e-02  1.59594999e-03  8.15214776e-03  1.27564324e-02
 -1.82663500e-02  2.43524555e-02  2.06144270e-03 -1.51197379e-02
  4.10767924e-03  6.10244693e-03 -1.19378436e-02  9.47892852e-03
  1.82248857e-02 -1.10979751e-03 -2.10657883e-02  1.19217988e-02
 -3.34069203e-03 -2.99571408e-03  5.04342047e-03 -1.54435504e-02
 -1.61644109e-02 -1.06337364e-03 -5.86290611e-03 -3.74967232e-03
  7.27828406e-03 -9.33383126e-03  8.70098919e-03  6.52185362e-03
  3.61848576e-03  4.98640351e-04  3.12484102e-03  1.19153401e-02
  2.15859171e-02 -4.32068063e-03  2.06008665e-02 -9.53370705e-03
 -3.29327420e-03  1.81369092e-02  1.13796694e-02 -2.67953537e-02
  1.39712729e-02 -9.59111378e-03  8.81327968e-03 -1.32999057e-03
  1.18830409e-02  2.72178557e-03 -1.71974767e-03  2.02868897e-02
  9.27997660e-03  7.85296876e-03  9.14768409e-03 -1.22201657e-02
  4.93466482e-03 -2.73190765e-03  1.84215081e-04  1.18957944e-02
 -7.30011519e-03

<ipython-input-10-11378e74a7b1>:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('1')
<ipython-input-10-11378e74a7b1>:16: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs['1'])


In [11]:
def output_embeddings(output_path, albums_path):
    embeddings = {}
    
    for dirName, subdirList, fileList in os.walk(albums_path, topdown=False):
        album_lyrics = ''
        album_name = dirName.split('/')[-1]
        
        for fname in fileList:
            
            if fname.endswith('.txt'):
                lyrics_file = open(dirName + '/' + fname, "r")
                lyrics = lyrics_file.read()
                lyrics_file.close()
                
                
                lyrics = lyrics.replace('\n', ' ')
                lyrics = lyrics.replace('\\', '')
                lyrics = lyrics.strip(' ')
                lyrics = re.sub(r'\[[^\]]*\]', '', lyrics)
                album_lyrics = album_lyrics + lyrics.lower() + ' '
        
        if not len(album_lyrics) == 0:
            temp = word_tokenize(album_lyrics)
            vector = model.infer_vector(temp)
            embeddings[album_name.replace('/','-')] = vector
    
    return embeddings

In [12]:
embeddings = output_embeddings(output_path, albums_path)

#print(embeddings)

In [13]:
file = "new_embeddings"

file_object = open(file,'wb')

pickle.dump(embeddings, file_object)

file_object.close()